##Xây dựng model nhận dạng ký tự mã Morse
Bài tập lớn: Máy học cơ bản và ứng dụng

Nhóm 10_L01_HK231

Tô Bạch Long_2011559

Tạ Thị Nhã Linh_ 2013642


##Giải nén thư mục và thêm thư viện

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

MessageError: ignored

In [ ]:
! mkdir '/content/dataset'

In [ ]:
# %%capture
! unzip '/content/gdrive/MyDrive/ML_231/morse-dataset-master.zip' -d '/content/dataset'

In [ ]:
import math
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt
import tensorflow as tf
import librosa
import os
import re
import pickle
import json
from keras.models import Sequential
from keras.layers import Dense
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Input, Dense, LSTM, Flatten, Conv2D
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

##Phân tập dữ liệu

In [ ]:
def load_data(filename):
    '''
    General case to load any data from filename
    Output tuple:
        xtr, ytr : Data and labels for training
        xva, yva : Data and labels for validation
        xte, yte : Data and labels for test
    '''
    loaded = np.load(filename)
    xtr = loaded['xtr']
    ytr = loaded['ytr']
    xva = loaded['xva']
    yva = loaded['yva']
    xte = loaded['xte']
    yte = loaded['yte']
    return (xtr, ytr, xva, yva, xte, yte)

In [ ]:
filename = '/content/dataset/morse-dataset-master/baseline.npz'

In [ ]:
xtrain, ytrain, xval, yval, xtest, ytest = load_data(filename = '/content/dataset/morse-dataset-master/baseline.npz')

In [ ]:
ytrain[1]

In [ ]:
# Kích thước ảnh đầu ra
width = 64
height = 1
# random_image = random.choice(X_val)
# print(class_names[Y_val[random_image.]])
random_index = np.random.randint(0, len(xtest))
random_image = xtest[random_index]
true_label = ytest[random_index]
# Vector đầu vào
vector = random_image  # Giá trị của vector tùy thuộc vào yêu cầu của bạn

# Chuyển đổi vector thành mảng numpy
array = np.array(vector)

# Điều chỉnh kích thước mảng thành kích thước ảnh mong muốn
array = np.resize(array, (height, width))

# Hiển thị ảnh sử dụng matplotlib
plt.imshow(array, cmap='gray')
plt.axis('off')
plt.show()
print(true_label)
for i in range(64):
  if (true_label[i] == 1):
        print(i)

Trích ký tự và mã Morse tương ứng trong data

In [ ]:
Codebook = np.load('/content/dataset/morse-dataset-master/Codebook.npy', allow_pickle=True).item()

In [ ]:
Codebook

In [ ]:
#label
label = Codebook.keys()
label = list(label)

In [ ]:
label[4]

In [ ]:
#Mã Morse tương ứng
data_new = Codebook.values()
data_new = list(data_new)

In [ ]:
data_new

##Xây dựng mô hình

In [ ]:
xtrain, xval = xtrain/255.0, xval/255.0

In [ ]:
model = Sequential()
model.add(Dense(64, activation='relu', input_shape=(64,)))
model.add(Dense(32, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(64, activation='sigmoid'))  # Điều chỉnh số lượng đầu ra thành 64

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
history = model.fit(xtrain, ytrain, epochs=10, validation_data=(xval, yval))

In [ ]:
# # Lấy độ chính xác tốt nhất từ history cũ
# best_accuracy = max(history.history['val_accuracy'])

# # Chạy một lần huấn luyện mới với độ chính xác tốt nhất
# new_history = model.fit(xtrain, ytrain, epochs=10, validation_data=(xval, yval))

In [ ]:
# # Lấy độ chính xác tốt nhất từ history cũ
# best_accuracy = max(new_history.history['val_accuracy'])

# # Chạy một lần huấn luyện mới với độ chính xác tốt nhất
# new_history = model.fit(xtrain, ytrain, epochs=10, validation_data=(xval, yval))

In [ ]:
# # Lấy độ chính xác tốt nhất từ history cũ
# best_accuracy = max(new_history.history['val_accuracy'])

# # Chạy một lần huấn luyện mới với độ chính xác tốt nhất
# history = model.fit(xtrain, ytrain, epochs=10, validation_data=(xval, yval))

In [ ]:
#Get loss information
train_loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1,11) #1...  31

#show history
plt.figure(figsize = (10,6))
plt.plot (epochs, train_loss, 'bo-', label='Training Loss')
plt.plot (epochs, val_loss, 'ro-', label='validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

In [ ]:
random_index = np.random.randint(0, len(xval))
random_image = xval[random_index]
true_label = yval[random_index]
random_image = random_image.reshape((1, 64))
for i in range(64):
      if true_label[i] == 1:
          nhan = i
plt.imshow(random_image, cmap='gray')
plt.axis('off')
plt.title(data_new[nhan])
plt.show()

print(true_label)
# 4 pixel trở lên -
#

In [ ]:
random_index = np.random.randint(0, len(xval))
random_image = xval[random_index]
true_label = yval[random_index]
random_image = random_image.reshape((1, 64))
for i in range(64):
      if true_label[i] == 1:
          nhan = i
# print(random_image.shape)
#make predictions
predictions = model.predict(random_image)
# print(predictions)

#get the predicted label
predicted_label = np.argmax(predictions)
# for i in range(64):
#       if predicted_label[i] == 1:
#           predict = i
# print(data_new[predicted_label])
#Plot the image
plt.figure(figsize=(8,8))
# plt.imshow(random_image[0] * 255.0, cmap='gray')
plt.imshow(random_image, cmap='gray')
plt.title(f'True Label: {data_new[nhan]}\nPredicted Label: {data_new[predicted_label]}')
plt.axis('off')
plt.show()

##Train với data mới

In [ ]:
import pickle
file_path = '/content/gdrive/MyDrive/ML/spectrogram_dataset.pkl'

# Đọc dữ liệu từ tệp tin
with open(file_path, 'rb') as file:
    spectrogram_dataset = pickle.load(file)

In [ ]:
file_path = '/content/gdrive/MyDrive/ML/tap_y.pkl'

# Đọc dữ liệu từ tệp tin
with open(file_path, 'rb') as file:
    tap_y = pickle.load(file)

In [ ]:
spectrogram_dataset

In [ ]:
# from sklearn.model_selection import train_test_split
# X_train, X_val, y_train, y_val = train_test_split(spectrogram_dataset, tap_y, test_size=0.2, random_state=42) #80% dữ liệu gán cho X_train

In [ ]:
#vẽ spectrogram
def plot_spectrogram(spectrogram, start=0, end=-1):
    plt.figure(figsize=(13, 4))
    librosa.display.specshow(spectrogram[:, start:end], sr=22050, x_axis='time', y_axis='mel')
    plt.colorbar(format='%+2.0f dB')
    plt.title('Spectrogram')
    plt.show()
threshold = -15
def silence_detection_num(spectrogram, show=False):
    mask_num = np.where(spectrogram < threshold, -80, -10)
    word_num = np.where(np.any(mask_num == -10, axis=0), '-', '.')
#     words = np.split(spectrogram, word_num, axis=1)

    if show:
        text = translate(word_num)

    return mask_num, text

def translate(symbols):
    symbols = ''.join(symbols)
    # Split the morse code into segments of consecutive dots or dashes
    segments = re.findall(r'\.+|-+', symbols)

    # Translate to Morse Code
    morse = ''
    for segment in segments:
        if '.' in segment:
            if len(segment) > 30:
                morse += '|'
            elif len(segment) > 5:
                morse += ' '
        elif '-' in segment:
            if len(segment) < 7:
                morse += '.'
            elif len(segment) >= 7:
                morse += '-'

    # Translate to Alphabetical Text
    ALPHA_TO_MORSE = { 'A':'.-', 'B':'-...',
                    'C':'-.-.', 'D':'-..', 'E':'.',
                    'F':'..-.', 'G':'--.', 'H':'....',
                    'I':'..', 'J':'.---', 'K':'-.-',
                    'L':'.-..', 'M':'--', 'N':'-.',
                    'O':'---', 'P':'.--.', 'Q':'--.-',
                    'R':'.-.', 'S':'...', 'T':'-',
                    'U':'..-', 'V':'...-', 'W':'.--',
                    'X':'-..-', 'Y':'-.--', 'Z':'--..',
                    '1':'.----', '2':'..---', '3':'...--',
                    '4':'....-', '5':'.....', '6':'-....',
                    '7':'--...', '8':'---..', '9':'----.',
                    '0':'-----', ', ':'--..--', '.':'.-.-.-',
                    '?':'..--..', '/':'-..-.', '-':'-....-',
                    '(':'-.--.', ')':'-.--.-', '=':'-...-'}

    MORSE_TO_ALPHA = {'.-': 'A', '-...': 'B',
                      '-.-.': 'C', '-..': 'D', '.': 'E',
                      '..-.': 'F', '--.': 'G', '....': 'H',
                      '..': 'I', '.---': 'J', '-.-': 'K',
                      '.-..': 'L', '--': 'M', '-.': 'N',
                      '---': 'O', '.--.': 'P', '--.-': 'Q',
                      '.-.': 'R', '...': 'S', '-': 'T',
                      '..-': 'U', '...-': 'V', '.--': 'W',
                      '-..-': 'X', '-.--': 'Y', '--..': 'Z',
                      '.----': '1', '..---': '2', '...--': '3',
                      '....-': '4', '.....': '5', '-....': '6',
                      '--...': '7', '---..': '8', '----.': '9',
                      '-----': '0', '--..--': ', ', '.-.-.-': '.',
                      '..--..': '?', '-..-.': '/', '-....-': '-',
                      '-.--.': '(', '-.--.-': ')', '-...-': '='}

    morse_string = morse.strip('|')
    words = morse_string.split('|')
    word_sequences = [word.split(' ') for word in words] # Output: [['-...-'], ['-.', '--', '.--']]

    text = ''
    for word in word_sequences:
        for char in word:
            try:
                text += MORSE_TO_ALPHA[char]
            except KeyError:
                text += '?'
        text += ' '

    return text

In [ ]:
tap_x = []
M = 50
for i in range(M):
    data, text = silence_detection_num(spectrogram_dataset[0], show=True)
    tap_x.append(data)
    # print(text)
    # plot_spectrogram(X_train, end=500)

In [ ]:
tap_y_new = tap_y[:50]

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(tap_x, tap_y_new, test_size=0.2, random_state=42)

In [ ]:
for j in range(len(X_train[1])):
  for i in range(len(X_train[2])):
      if X_train[30][j][i] != -80:
          print("j = ",j)
          # print("i = ",i)
          # (_, 27 28 29, _)

In [ ]:
X_train = np.array(X_train)
ma_tran = np.zeros((50, 1, 20232))
for k in range(50):
    ma_tran[k, 0, :] = np.copy(X_train[0, 28, :])/80 + 1

In [ ]:
ma_tran_1 = []
for i in range(len(ma_tran[0][0])):
  if ma_tran[0][0][i] != 0:
      num = i
      break
for k in range(num, len(ma_tran[0][0])):
   num_1 = ma_tran[0][0][k]
   ma_tran_1.append(num_1)
# ma_tran_1

In [ ]:
def division_symbols (ma_tran):
        ma_tran_1 = []
        for i in range(len(ma_tran)):
          if ma_tran[i] != 0:
              num = i
              break
        for k in range(num, len(ma_tran)):
          num_1 = ma_tran[k]
          ma_tran_1.append(num_1)
        matrix = np.zeros([1,64])
        symbols = []
        sample = []
        count = 0
        pf_idx = 0
       # pfrom = ma_tran[pf_idx]
       # pcurrent = pfrom
        for p_idx in range(pf_idx, len(ma_tran_1)):
            #  print(p_idx)
             #sample = matrix
             pto = ma_tran_1[p_idx]
             if pto!=0 :   # value != 0
                sample.append(pto)
             else:         # value = 0
                # pass
                count += 1
                if count < 40:
                   sample.append(0)
                else:
                   m = len(sample)
                  #  print(m)
                   for i in range(m,64) :
                        sample.append(0)
                   symbols.append(sample)

                  #  symbols.append(matrix)
                   sample = []

            #  break
                pf_idx = p_idx
            #  pfrom = ma_tran_1[p_idx]
            #  pcurrent = pfrom
        return symbols

In [ ]:
# ma_tran[0][0]
for value in ma_tran[0][0]:
    print(value)

In [ ]:
symbols_temp = division_symbols(ma_tran[0][0])

In [ ]:
symbols_temp[14]

In [ ]:
symbols_temp_1 = []
# rong = np.zeros([64, 1])
for i in range(len(symbols_temp)):
    if np.sum(symbols_temp[i]) != 0:
        symbols_temp_1.append(symbols_temp[i])

In [ ]:
for i in range(len(symbols_temp_1)):
        symbols_temp_1[i] = np.transpose(symbols_temp_1[i][:64])

In [ ]:
# i= 1
# plt.figure(figsize=(8,8))
# plt.imshow(symbols_temp_1[i], cmap='gray')
# plt.axis('off')
# plt.show()

In [ ]:
def predict(xval):
        random_image = xval.reshape((1, 64))
        predictions = model.predict(random_image)
        predicted_label = np.argmax(predictions)
        ki_tu_du_doan = label[predicted_label]
        return ki_tu_du_doan

In [ ]:
import json
predict_content = dict()
for i in range(len(symbols_temp_1)):
      predict_content[i] = predict(symbols_temp_1[i])
      plt.figure(figsize=(8,8))
      symbols_temp_1[i] = np.reshape(symbols_temp_1[i], (1, 64))
      plt.imshow(symbols_temp_1[i], cmap='gray')
      plt.axis('off')
      plt.show()
predict_content = ''.join(predict_content.values())
print("Văn bản dự đoán: ", predict_content)
print("Van ban dung: ", y_train[0])